# Python Environment Setup

In [80]:
!pip3 install virtualenv

In [81]:
!python3 -m venv myenv

In [82]:
!source myenv/bin/activate

# This code will create a spreadsheet for all input, correct output and system output

In [148]:
import json
import pandas as pd

## creating the necessary columns as required

In [149]:
column_names = ["query_type","relation", "query","correct output", "system output","ssg output","Exact Match", "Soft Match"]
my_data = pd.DataFrame(columns = column_names)

In [150]:
query_type = []
relation = []
query = []
correct = []
system = []
exact = []
soft = []
ssg_op = []


## This method will get query, answer, relation and query type from test file.

In [151]:
def get_data_from_original(path):
    with open(path) as json_file:
        for line in json_file:
            db = json.loads(line)
            for all_query in db["queries"]:
                query.append(all_query['query'])
                correct.append(all_query['answer'])
                relation.append(all_query['relation'])
                if "arg" in all_query['type'] or "min" in all_query['type'] or "max" in all_query['type']:
                    query_type.append("min/max")
                else:
                    query_type.append(all_query['type'])



In [152]:
get_data_from_original("onlytable/balanced_test.jsonl")

## This function will get the gold facts and predicted facts from the predicted file

In [153]:
def get_data_from_prediction(path):
    with open(path) as json_file:
        db = json.load(json_file)

        for idx,data in enumerate(db):
            temp_ssg = []
            ssg_output = [[f[0] for f in ss] for ss in data["ssg_output"]]
            temp_soft = 0
            temp_exact = 0
            for s in ssg_output:
                if len(s)==0 and data["gold_facts"] ==[]:
                    temp_exact+=1

                if s in data['gold_facts']:

                    temp_exact+=1

                else:
                    if len(s) > 1 and [s[1], s[0]] in data['gold_facts']:
                        temp_exact+=1
                    else:
                        for gold_s in data['gold_facts']:
                            if set(gold_s) <= set(s):
                                temp_soft+=1
                                break
            if len(data['gold_facts']) == temp_exact:
                ssg_op.append(data["ssg_output"])
                system.append(data['gold_facts'])
                exact.append(1)
                soft.append(1)
            elif temp_exact>1:
                ssg_op.append(data["ssg_output"])
                system.append(data['gold_facts'])
                exact.append(0)
                soft.append(1)
            elif temp_soft>=1:
                ssg_op.append(data["ssg_output"])
                system.append(data['gold_facts'])
                exact.append(0)
                soft.append(1)
            else:
                ssg_op.append(data["ssg_output"])
                system.append("None")
                exact.append(0)
                soft.append(0)


In [154]:
get_data_from_prediction("onlytable/balanced_test_0.7_ssg_sup.json")

In [155]:
my_data["query_type"] = query_type
my_data['query'] = query
my_data['relation'] = relation
my_data["correct output"] = correct
my_data["system output"] = system
my_data['Exact Match'] = exact
my_data["Soft Match"] = soft
my_data["ssg output"] = ssg_op

In [156]:
my_data.head()

,query_type,relation,query,correct output,system output,ssg output,Exact Match,Soft Match
0,set,P54,Who is a member of United States?,[],None,[[]],0,0
1,min/max,P54,Who has played for the fewest teams?,[Ýokary Liga],"[[1], [2], [3], [4]]","[[[2, FIFA World Cup team Belgium]], [[3, FIFA...",0,1
2,set,P54,List teams FIFA World Cup plays for,"[Argentina, Portugal, Brazil, Belgium]","[[2], [3], [6], [7, 8]]","[[[2, FIFA World Cup team Belgium]], [[3, FIFA...",0,1
3,set,P54,List teams FIFA World Cup plays for,"[Portugal, Brazil, Belgium]","[[2], [3], [6]]","[[[2, FIFA World Cup team Belgium]], [[3, FIFA...",1,1
4,bool,P54,Does FIFA World Cup play for Argentina?,[],[],"[[[2, FIFA World Cup team Belgium]], [[3, FIFA...",1,1


## Save the dataframe as spreadsheet

In [157]:
my_data.to_csv('onlytable/onlytable_sheet_test.tsv',sep='\t', index=False)